# <center>Scraping twitch tracker for the top 500 streamers for the top 10 games</center>
***

In [1]:
import pandas as pd
import numpy as np
import bs4
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

pd.options.display.float_format = '{:.0f}'.format
chrome_options = Options()
chrome_options.add_argument('--headless')
path_to_driver = '/usr/bin/chromedriver'
chrome = webdriver.Chrome(path_to_driver, options=chrome_options)

In [2]:
def save_df_as_csv(df):
    import os
    path = r'./data/top_streamers.csv'
    
    if not os.path.exists(r'./data'):
        os.mkdir(r'./data')
    
    if os.path.isfile(path):
        os.remove(path)
        df.to_csv(path, index=False)
        return 'file overriden'
    else:
        df.to_csv(path, index=False)
        return 'saved'

In [3]:
def load_csv(path):
    import os
    if not os.path.isfile(path):
        raise Exception('path does not exist')
    return pd.read_csv(path)

In [4]:
def get_top_n_most_subscribed_streamers(driver=None, n=10):
    if driver is None:
        raise Exception('You have to provide a web driver')
        
    headers = ['#', 'streamer_name']
    result = []
    url = 'https://twitchtracker.com/subscribers?page='
    for i in range(n):
        print('extracting page ' + str(i+1))
        driver.get(url + str(i+1))
        html = driver.execute_script('return document.body.innerHTML;')
        soup = bs(html, 'lxml')
        for j in range(21):
            if j+1 == 11:
                continue
            temp = []
            streamer_position = soup.table.find_all('tr')[j+1].find_all('td')[0].text.replace('#', '')
            streamer_name = soup.table.find_all('tr')[j+1].find_all('td')[3].a.text
            temp.append([streamer_position, streamer_name])
            result.append(temp)
        time.sleep(3)
    result = np.array(result)
    result = result.squeeze()
    return pd.DataFrame(result, columns=headers)

In [39]:
def get_streamer_names(driver=None, names=''):
    if driver is None:
        raise Exception('You have to provide a web driver')
        
    headers = ['streamer_name', 'hours_streamed', 'average_viewers', 
               'peak_viewers', 'hours_watched', 'followers_gained', 
               'followers_per_hour', 'viewers_gained', 'active_days']
    result = []
    url = 'https://twitchtracker.com/'
    
    for name in names:
        if name == 'RanbooLive':
            print('my')
            break
        driver.get(url + name)
        month_button = driver.find_element_by_xpath('//*[@id="select-performance"]/span[2]')
        month_button.click()
        html = driver.execute_script('return document.body.innerHTML;')
        soup = bs(html, 'lxml')
        
        result.append([name])
        for i in range(len(headers)):
            temp = []
            if i < 7:
                data = soup.find_all('div', {'id': 'performance-panel'})[0].find_all('div', {'class': 'g-x-s-value g-x-s-contrast'})[i].find_all('span')[0].text
                temp.append(data)
            elif i == 7:
                print('ass')
                active_days = soup.find_all('div', {'id': 'performance-panel'})[0].find_all('div', {'class': 'g-x-s-value g-x-s-contrast'})[i].find_all('span')[0].text.split('/')[0]
                temp.append(active_days)
            result.append(temp)
        # soup.find_all('div', {'id': 'performance-panel'})[0].find_all('div', {'class': 'g-x-s-value g-x-s-contrast'})[2].find_all('span')[0].text
    print(np.array(result)[:-1].reshape(1, 9))
    return pd.DataFrame(np.array(result).squeeze()[:-1].reshape(1, 9), columns=headers)

In [40]:
path = r'./data/top_streamers.csv'
#df = get_top_n_most_subscribed_streamers(driver=chrome, n=1)
#save_df_as_csv(df)
df = load_csv(path)
streamer_names = np.array(df['streamer_name'])
get_streamer_names(chrome, streamer_names)

ass
my
[[list(['ludwig']) list(['620']) list(['40,744']) list(['96,602'])
  list(['25,263,016']) list(['851,015']) list(['1,372'])
  list(['30,580,688']) list(['29'])]]


/home/mykull/miniconda3/envs/twitch-scrape/lib/python3.7/site-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/mykull/miniconda3/envs/twitch-scrape/lib/python3.7/site-packages/ipykernel_launcher.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


,streamer_name,hours_streamed,average_viewers,peak_viewers,hours_watched,followers_gained,followers_per_hour,viewers_gained,active_days
0,[ludwig],[620],"[40,744]","[96,602]","[25,263,016]","[851,015]","[1,372]","[30,580,688]",[29]
